### Adquisición de Datos & Merges


### Funciones

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

def data_profiling(df, output_file):
    # Opciones para que sea liviano
    profile = ProfileReport(
        df.sample(20000, random_state=42) if len(df) > 20000 else df,
        title=output_file,
        minimal=True,         # desactiva análisis costosos
        explorative=True      # agrega secciones útiles
    )

    profile.to_file(output_file)  # <-- abre este HTML en el navegador

In [2]:
# Funcion para mostrar un resumen del dataframe
def df_info_summary(df: pd.DataFrame):
    total = len(df)
    non_null = df.notnull().sum()
    nulls = df.isnull().sum()
    dtypes = df.dtypes
    
    resumen = pd.DataFrame({
        "Non-Null Count": non_null,
        "Null Count": nulls,
        "% Null": (nulls / total * 100).round(2),
        "Dtype": dtypes
    })
    print(resumen)

### Importamos bureau.csv & bureau_balance.csv

In [3]:
# Create data profiles for bureau and bureau_balance datasets

bureau_df = pd.read_csv("bureau.csv")
bureau_balance_df = pd.read_csv("bureau_balance.csv")


In [4]:
df_info_summary(bureau_df)
df_info_summary(bureau_balance_df)

                        Non-Null Count  Null Count  % Null    Dtype
SK_ID_CURR                     1716428           0    0.00    int64
SK_ID_BUREAU                   1716428           0    0.00    int64
CREDIT_ACTIVE                  1716428           0    0.00   object
CREDIT_CURRENCY                1716428           0    0.00   object
DAYS_CREDIT                    1716428           0    0.00    int64
CREDIT_DAY_OVERDUE             1716428           0    0.00    int64
DAYS_CREDIT_ENDDATE            1610875      105553    6.15  float64
DAYS_ENDDATE_FACT              1082775      633653   36.92  float64
AMT_CREDIT_MAX_OVERDUE          591940     1124488   65.51  float64
CNT_CREDIT_PROLONG             1716428           0    0.00    int64
AMT_CREDIT_SUM                 1716415          13    0.00  float64
AMT_CREDIT_SUM_DEBT            1458759      257669   15.01  float64
AMT_CREDIT_SUM_LIMIT           1124648      591780   34.48  float64
AMT_CREDIT_SUM_OVERDUE         1716428          

Bureau data join

In [5]:
# Crear tabla de conteo de status por SK_ID_BUREAU
# OHE con get_dummies
status_dummies = pd.get_dummies(bureau_balance_df["STATUS"], prefix="SUM_STATUS", dtype="int32")

# Concatenar con SK_ID_BUREAU
tmp = pd.concat([bureau_balance_df[["SK_ID_BUREAU"]], status_dummies], axis=1)

# Agrupar por SK_ID_BUREAU y sumar
status_counts =tmp.groupby("SK_ID_BUREAU", as_index=False).sum()

In [6]:
len(status_counts)
df_info_summary(status_counts)

              Non-Null Count  Null Count  % Null  Dtype
SK_ID_BUREAU          817395           0     0.0  int64
SUM_STATUS_0          817395           0     0.0  int32
SUM_STATUS_1          817395           0     0.0  int32
SUM_STATUS_2          817395           0     0.0  int32
SUM_STATUS_3          817395           0     0.0  int32
SUM_STATUS_4          817395           0     0.0  int32
SUM_STATUS_5          817395           0     0.0  int32
SUM_STATUS_C          817395           0     0.0  int32
SUM_STATUS_X          817395           0     0.0  int32


In [7]:
# Join final de tablas bureau y bureau_balance agregando los conteos de status
bureau_df_join = bureau_df.merge(status_counts, on="SK_ID_BUREAU", how="left")

# Rellenar NaN con 0 y casteo a int32
status_cols = [col for col in bureau_df_join.columns if col.startswith("SUM_STATUS_")]
bureau_df_join[status_cols] = bureau_df_join[status_cols].fillna(0).astype("int32")

In [8]:
# OHE con get_dummies
to_keep_numeric_and_boolean = bureau_df_join.select_dtypes(include=["number", "bool"]).columns.tolist()
to_do_ohe = ['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']
bureau_df_join = bureau_df_join[to_keep_numeric_and_boolean + to_do_ohe].copy()
bureau_df_categorical = pd.get_dummies(bureau_df_join[to_do_ohe], dtype="int32")
bureau_df_join = pd.concat([bureau_df_join[to_keep_numeric_and_boolean], bureau_df_categorical], axis=1)


In [9]:
# Definir el diccionario de agregación con renombrado
agg_dict = {
    # Columnas con media y renombrado
    'DAYS_CREDIT': [('DAYS_CREDIT_MEAN', 'mean')],
    'DAYS_CREDIT_ENDDATE': [('DAYS_CREDIT_ENDDATE_MEAN', 'mean')],
    'DAYS_ENDDATE_FACT': [('DAYS_ENDDATE_FACT_MEAN', 'mean')],
    'AMT_CREDIT_MAX_OVERDUE': [('AMT_CREDIT_MAX_OVERDUE_MEAN', 'mean')],
    'AMT_CREDIT_SUM': [('AMT_CREDIT_SUM_MEAN', 'mean')],
    'AMT_CREDIT_SUM_DEBT': [('AMT_CREDIT_SUM_DEBT_MEAN', 'mean')],
    'AMT_CREDIT_SUM_LIMIT': [('AMT_CREDIT_SUM_LIMIT_MEAN', 'mean')],
    'AMT_CREDIT_SUM_OVERDUE': [('AMT_CREDIT_SUM_OVERDUE_MEAN', 'mean')],
    'DAYS_CREDIT_UPDATE': [('DAYS_CREDIT_UPDATE_MEAN', 'mean')],
    'AMT_ANNUITY': [('AMT_ANNUITY_MEAN', 'mean')],
    
    # Columnas con suma y renombrado
    'CREDIT_DAY_OVERDUE': [('CREDIT_DAY_OVERDUE_SUM', 'sum')],
    'CNT_CREDIT_PROLONG': [('CNT_CREDIT_PROLONG_SUM', 'sum')],
    'SUM_STATUS_0': [('MONTHS_WITH_STATUS_DPD_0_SUM', 'sum')],
    'SUM_STATUS_1': [('MONTHS_WITH_STATUS_DPD_1to30_SUM', 'sum')],
    'SUM_STATUS_2': [('MONTHS_WITH_STATUS_DPD_31to60_SUM', 'sum')],
    'SUM_STATUS_3': [('MONTHS_WITH_STATUS_DPD_61to90_SUM', 'sum')],
    'SUM_STATUS_4': [('MONTHS_WITH_STATUS_DPD_91to120_SUM', 'sum')],
    'SUM_STATUS_5': [('MONTHS_WITH_STATUS_DPD_over120_SUM', 'sum')],
    'SUM_STATUS_C': [('MONTHS_WITH_STATUS_CLOSED_SUM', 'sum')],
    'SUM_STATUS_X': [('MONTHS_WITH_STATUS_UNKNOWN_SUM', 'sum')],
    'CREDIT_ACTIVE_Active': [('CREDIT_ACTIVE_Active_SUM', 'sum')],
    'CREDIT_ACTIVE_Bad debt': [('CREDIT_ACTIVE_Bad debt_SUM', 'sum')],
    'CREDIT_ACTIVE_Closed': [('CREDIT_ACTIVE_Closed_SUM', 'sum')],
    'CREDIT_ACTIVE_Sold': [('CREDIT_ACTIVE_Sold_SUM', 'sum')],
    'CREDIT_CURRENCY_currency 1': [('CREDIT_CURRENCY_currency 1_SUM', 'sum')],
    'CREDIT_CURRENCY_currency 2': [('CREDIT_CURRENCY_currency 2_SUM', 'sum')],
    'CREDIT_CURRENCY_currency 3': [('CREDIT_CURRENCY_currency 3_SUM', 'sum')],
    'CREDIT_CURRENCY_currency 4': [('CREDIT_CURRENCY_currency 4_SUM', 'sum')],
    'CREDIT_TYPE_Another type of loan': [('CREDIT_TYPE_Another type of loan_SUM', 'sum')],
    'CREDIT_TYPE_Car loan': [('CREDIT_TYPE_Car loan_SUM', 'sum')],
    'CREDIT_TYPE_Cash loan (non-earmarked)': [('CREDIT_TYPE_Cash loan (non-earmarked)_SUM', 'sum')],
    'CREDIT_TYPE_Consumer credit': [('CREDIT_TYPE_Consumer credit_SUM', 'sum')],
    'CREDIT_TYPE_Credit card': [('CREDIT_TYPE_Credit card_SUM', 'sum')],
    'CREDIT_TYPE_Interbank credit': [('CREDIT_TYPE_Interbank credit_SUM', 'sum')],
    'CREDIT_TYPE_Loan for business development': [('CREDIT_TYPE_Loan for business development_SUM', 'sum')],
    'CREDIT_TYPE_Loan for purchase of shares (margin lending)': [('CREDIT_TYPE_Loan for purchase of shares (margin lending)_SUM', 'sum')],
    'CREDIT_TYPE_Loan for the purchase of equipment': [('CREDIT_TYPE_Loan for the purchase of equipment_SUM', 'sum')],
    'CREDIT_TYPE_Loan for working capital replenishment': [('CREDIT_TYPE_Loan for working capital replenishment_SUM', 'sum')],
    'CREDIT_TYPE_Microloan': [('CREDIT_TYPE_Microloan_SUM', 'sum')],
    'CREDIT_TYPE_Mobile operator loan': [('CREDIT_TYPE_Mobile operator loan_SUM', 'sum')],
    'CREDIT_TYPE_Mortgage': [('CREDIT_TYPE_Mortgage_SUM', 'sum')],
    'CREDIT_TYPE_Real estate loan': [('CREDIT_TYPE_Real estate loan_SUM', 'sum')],
    'CREDIT_TYPE_Unknown type of loan': [('CREDIT_TYPE_Unknown type of loan_SUM', 'sum')]
}

# Aplanar el diccionario para la agregación
flat_agg_dict = {}
for col, operations in agg_dict.items():
    for new_name, func in operations:
        flat_agg_dict[new_name] = (col, func)

# Aplicar la agregación con renombrado ### ME TIRA ERROR
bureau_df_agg = bureau_df_join.groupby(['SK_ID_CURR']).agg(**flat_agg_dict).reset_index()

# Mostrar las primeras filas
bureau_df_agg.head()

,SK_ID_CURR,DAYS_CREDIT_MEAN,DAYS_CREDIT_ENDDATE_MEAN,DAYS_ENDDATE_FACT_MEAN,AMT_CREDIT_MAX_OVERDUE_MEAN,AMT_CREDIT_SUM_MEAN,AMT_CREDIT_SUM_DEBT_MEAN,AMT_CREDIT_SUM_LIMIT_MEAN,AMT_CREDIT_SUM_OVERDUE_MEAN,DAYS_CREDIT_UPDATE_MEAN,AMT_ANNUITY_MEAN,CREDIT_DAY_OVERDUE_SUM,CNT_CREDIT_PROLONG_SUM,MONTHS_WITH_STATUS_DPD_0_SUM,MONTHS_WITH_STATUS_DPD_1to30_SUM,MONTHS_WITH_STATUS_DPD_31to60_SUM,MONTHS_WITH_STATUS_DPD_61to90_SUM,MONTHS_WITH_STATUS_DPD_91to120_SUM,MONTHS_WITH_STATUS_DPD_over120_SUM,MONTHS_WITH_STATUS_CLOSED_SUM,MONTHS_WITH_STATUS_UNKNOWN_SUM,CREDIT_ACTIVE_Active_SUM,CREDIT_ACTIVE_Bad debt_SUM,CREDIT_ACTIVE_Closed_SUM,CREDIT_ACTIVE_Sold_SUM,CREDIT_CURRENCY_currency 1_SUM,CREDIT_CURRENCY_currency 2_SUM,CREDIT_CURRENCY_currency 3_SUM,CREDIT_CURRENCY_currency 4_SUM,CREDIT_TYPE_Another type of loan_SUM,CREDIT_TYPE_Car loan_SUM,CREDIT_TYPE_Cash loan (non-earmarked)_SUM,CREDIT_TYPE_Consumer credit_SUM,CREDIT_TYPE_Credit card_SUM,CREDIT_TYPE_Interbank credit_SUM,CREDIT_TYPE_Loan for business development_SUM,CREDIT_TYPE_Loan for purchase of shares (margin lending)_SUM,CREDIT_TYPE_Loan for the purchase of equipment_SUM,CREDIT_TYPE_Loan for working capital replenishment_SUM,CREDIT_TYPE_Microloan_SUM,CREDIT_TYPE_Mobile operator loan_SUM,CREDIT_TYPE_Mortgage_SUM,CREDIT_TYPE_Real estate loan_SUM,CREDIT_TYPE_Unknown type of loan_SUM
0,100001,-735.000000,82.428571,-825.500000,NaN,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143,0,0,31,1,0,0,0,0,110,30,3,0,4,0,7,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
1,100002,-874.000000,-349.000000,-697.500000,1681.029,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000,0,0,45,27,0,0,0,0,23,15,2,0,6,0,8,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0
2,100003,-1400.750000,-544.500000,-1097.333333,0.000,254350.125000,0.000000,202500.00000,0.0,-816.000000,NaN,0,0,0,0,0,0,0,0,0,0,1,0,3,0,4,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0
3,100004,-867.000000,-488.500000,-532.500000,0.000,94518.900000,0.000000,0.00000,0.0,-532.000000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
4,100005,-190.666667,439.333333,-123.000000,0.000,219042.000000,189469.500000,0.00000,0.0,-54.333333,1420.500000,0,0,14,0,0,0,0,0,5,2,2,0,1,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0


Importamos datos de previus_application

In [10]:
previous_application_df = pd.read_csv("previous_application.csv")

# OHE de columnas categóricas
# Identificar columnas categóricas
cat_cols = previous_application_df.select_dtypes(include=["object"]).columns.tolist()
print("Columnas categóricas:", cat_cols)

# Aplicar One Hot Encoding
previous_application_ohe = pd.get_dummies(previous_application_df, columns=cat_cols, dummy_na=True)

# Sumarizar previous_application_ohe por SK_ID_CURR para tener una fila por cada préstamo corriente
# Separar columnas según tipo
bool_cols = previous_application_ohe.select_dtypes(include=["bool"]).columns.tolist()
num_cols  = previous_application_ohe.select_dtypes(exclude=["bool"]).columns.tolist()

# Definir reglas de agregación
agg_dict = {col: "sum" for col in bool_cols}
agg_dict.update({col: "mean" for col in num_cols if col not in ["SK_ID_PREV", "SK_ID_CURR"]})

# Aplicar groupby
previous_application_agg = (
    previous_application_ohe
    .groupby("SK_ID_CURR")
    .agg(agg_dict)
    .reset_index()
)

Columnas categóricas: ['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']


Importamos datos de pos_cash_balance

In [11]:
pos_cash_balance_df = pd.read_csv("POS_CASH_balance.csv")

pos_cash_balance_df = pos_cash_balance_df.fillna(0)

# OHE con get_dummies
status_dummies = pd.get_dummies(pos_cash_balance_df["NAME_CONTRACT_STATUS"], prefix="STATUS_", dtype="int32")

# Concatenar pos_cash_balance_df
pos_cash_balance_df = pd.concat([pos_cash_balance_df, status_dummies], axis=1)

In [12]:
# Sumarizar pos_cash_balance_df por SK_ID_CURR para tener una fila por cada préstamo corriente
# Definir el diccionario de agregación según la columna
agg_dict = {
    # Columnas con media y renombrado
    'CNT_INSTALMENT': [('PCB_CNT_INSTALMENT_MEAN', 'mean')], # PCB_ para identificar que es de pos cash balance
    'CNT_INSTALMENT_FUTURE': [('PCB_CNT_INSTALMENT_FUTURE_MEAN', 'mean')],
    'SK_DPD': [('PCB_SK_DPD_MEAN', 'mean')],
    'SK_DPD_DEF': [('PCB_SK_DPD_DEF_MEAN', 'mean')],

    # Columnas con suma y renombrado
    'STATUS__Active': [('PCB_MONTHS_WITH_STATUS_Active_SUM', 'sum')],
    'STATUS__Amortized debt': [('PCB_MONTHS_WITH_STATUS_Amortized_debt_SUM', 'sum')],
    'STATUS__Approved': [('PCB_MONTHS_WITH_STATUS_Approved_SUM', 'sum')],
    'STATUS__Canceled': [('PCB_MONTHS_WITH_STATUS_Canceled_SUM', 'sum')],
    'STATUS__Completed': [('PCB_MONTHS_WITH_STATUS_Completed_SUM', 'sum')],
    'STATUS__Demand': [('PCB_MONTHS_WITH_STATUS_Demand_SUM', 'sum')],
    'STATUS__Returned to the store': [('PCB_MONTHS_WITH_STATUS_Returned_to_the_store_SUM', 'sum')],
    'STATUS__Signed': [('PCB_MONTHS_WITH_STATUS_Signed_SUM', 'sum')],
    'STATUS__XNA': [('PCB_MONTHS_WITH_STATUS_XNA_SUM', 'sum')]
}

# Aplanar el diccionario para la agregación
flat_agg_dict = {}
for col, operations in agg_dict.items():
    for new_name, func in operations:
        flat_agg_dict[new_name] = (col, func)

# Aplicar la agregación con renombrado
pos_cash_balance_df_agg = pos_cash_balance_df.groupby(['SK_ID_CURR']).agg(**flat_agg_dict).reset_index()

# Mostrar las primeras filas
pos_cash_balance_df_agg.head()

,SK_ID_CURR,PCB_CNT_INSTALMENT_MEAN,PCB_CNT_INSTALMENT_FUTURE_MEAN,PCB_SK_DPD_MEAN,PCB_SK_DPD_DEF_MEAN,PCB_MONTHS_WITH_STATUS_Active_SUM,PCB_MONTHS_WITH_STATUS_Amortized_debt_SUM,PCB_MONTHS_WITH_STATUS_Approved_SUM,PCB_MONTHS_WITH_STATUS_Canceled_SUM,PCB_MONTHS_WITH_STATUS_Completed_SUM,PCB_MONTHS_WITH_STATUS_Demand_SUM,PCB_MONTHS_WITH_STATUS_Returned_to_the_store_SUM,PCB_MONTHS_WITH_STATUS_Signed_SUM,PCB_MONTHS_WITH_STATUS_XNA_SUM
0,100001,4.000000,1.444444,0.777778,0.777778,7,0,0,0,2,0,0,0,0
1,100002,24.000000,15.000000,0.000000,0.000000,19,0,0,0,0,0,0,0,0
2,100003,10.107143,5.785714,0.000000,0.000000,26,0,0,0,2,0,0,0,0
3,100004,3.750000,2.250000,0.000000,0.000000,3,0,0,0,1,0,0,0,0
4,100005,10.636364,6.545455,0.000000,0.000000,9,0,0,0,1,0,0,1,0


Importamos datos de installments_payments

In [13]:
installments_payments_df = pd.read_csv("installments_payments.csv")

# Info sobre las columnas del installments_payments_df
"""
Histórico de pagos de cuotas de créditos anteriores.
•	SK_ID_PREV → ID del crédito previo.
•	SK_ID_CURR → ID del préstamo actual.
•	NUM_INSTALMENT_VERSION → Versión del calendario de pagos (0 si es tarjeta de crédito). Un cambio significa renegociación.
•	NUM_INSTALMENT_NUMBER → Número de cuota (1, 2, 3, …).
•	DAYS_INSTALMENT → Día en que debía pagarse la cuota (relativo a aplicación actual).
•	DAYS_ENTRY_PAYMENT → Día en que efectivamente se pagó (NaN si no se pagó).
•	AMT_INSTALMENT → Monto esperado de la cuota.
•	AMT_PAYMENT → Monto realmente pagado.
"""
df_info_summary(installments_payments_df)

# DAYS_ENTRY_PAYMENT NaN -> No se abonaron esas cuotas

                        Non-Null Count  Null Count  % Null    Dtype
SK_ID_PREV                    13605401           0    0.00    int64
SK_ID_CURR                    13605401           0    0.00    int64
NUM_INSTALMENT_VERSION        13605401           0    0.00  float64
NUM_INSTALMENT_NUMBER         13605401           0    0.00    int64
DAYS_INSTALMENT               13605401           0    0.00  float64
DAYS_ENTRY_PAYMENT            13602496        2905    0.02  float64
AMT_INSTALMENT                13605401           0    0.00  float64
AMT_PAYMENT                   13602496        2905    0.02  float64


In [14]:
"""
Por lo visto, los NaN en DAYS_ENTRY_PAYMENT y AMT_PAYMENT indican cuotas no pagadas. 
Vamos a crear las siguientes columnas:
- instalments_unpaid: cantidad de cuotas no abonadas (DAYS_ENTRY_PAYMENT no NaN)
- instalments_partially_paid: cantidad de cuotas abonadas parcialmente (DAYS_ENTRY_PAYMENT no NaN y AMT_PAYMENT < AMT_INSTALMENT)
- instalments_overdue: cantidad de cuotas vencidas y no pagadas (DAYS_ENTRY_PAYMENT NaN y DAYS_INSTALMENT < 0)
- amount debt: monto adeudado (AMT_INSTALMENT - AMT_PAYMENT) si no se pagó y si se pagó menos de lo debido
- dpd: días de atraso en el pago:
    - Si se pagó (DAYS_ENTRY_PAYMENT no es NaN): max(0, DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT)
    - Si no se pagó y está vencido: abs(DAYS_INSTALMENT) si DAYS_INSTALMENT < 0
"""

installments_payments_df["instalments_unpaid"] = installments_payments_df["DAYS_ENTRY_PAYMENT"].isna().astype(int)

installments_payments_df["instalments_partially_paid"] = (
    (~installments_payments_df["DAYS_ENTRY_PAYMENT"].isna()) & 
    (installments_payments_df["AMT_PAYMENT"] < installments_payments_df["AMT_INSTALMENT"]) &
    (installments_payments_df["AMT_PAYMENT"] > 0)
).astype(int)

installments_payments_df["instalments_overdue"] = ((installments_payments_df["DAYS_ENTRY_PAYMENT"].isna()) & (installments_payments_df["DAYS_INSTALMENT"] < 0)).astype(int)

# Calcular monto adeudado
installments_payments_df["amount_debt"] = installments_payments_df.apply(
    # lambda row: (row["AMT_INSTALMENT"] - row["AMT_PAYMENT"]) if (pd.notna(row["AMT_PAYMENT"]) or row["AMT_PAYMENT"] < row["AMT_INSTALMENT"]) else 0,
    # axis=1
    lambda row: max(0, row["AMT_INSTALMENT"] - row["AMT_PAYMENT"]) if pd.notna(row["AMT_PAYMENT"]) 
    else abs(row["AMT_INSTALMENT"]), 
    axis=1
).round(2)

# Calcular dpd considerando ambos casos
installments_payments_df["dpd"] = installments_payments_df.apply(
    lambda row: max(0, row["DAYS_ENTRY_PAYMENT"] - row["DAYS_INSTALMENT"]) if pd.notna(row["DAYS_ENTRY_PAYMENT"]) 
    else abs(row["DAYS_INSTALMENT"]) if row["DAYS_INSTALMENT"] < 0 else 0, 
    axis=1
).astype(int)

installments_payments_df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,instalments_unpaid,instalments_partially_paid,instalments_overdue,amount_debt,dpd
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,0,0,0,0.00,0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0,0,0,0.00,0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0,0,0,0.00,0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,0,0,0,0.00,0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,0,1,0,4.45,17


In [15]:
# Removemos columnas con NaN ya reemplazadas con las nuevas columnas
installments_payments_df = installments_payments_df.drop(columns=['DAYS_ENTRY_PAYMENT', 'AMT_PAYMENT'])

# Pasamos NUM_INSTALMENT_VERSION a int32
installments_payments_df['NUM_INSTALMENT_VERSION'] = installments_payments_df['NUM_INSTALMENT_VERSION'].astype('int32')

In [16]:
# Sumarizar installments_payments_df por SK_ID_CURR para tener una fila por cada préstamo corriente
# Definir el diccionario de agregación según la columna
agg_dict = {
    # Columnas con maximo y renombrado
    'NUM_INSTALMENT_VERSION': [('IP_NUM_INSTALMENT_VERSION_MAX', 'max')],  # Para ver cual fue el maximo renegociación al que llego / IP_ para identificar que es de installments payments

    # Columnas con media y renombrado
    'dpd': [('IP_DPD_MEAN', 'mean')],
    'amount_debt': [('IP_AMOUNT_DEBT_MEAN', 'mean')],

    # Columnas con suma y renombrado
    'instalments_unpaid': [('IP_INSTALMENTS_UNPAID_SUM', 'sum')],
    'instalments_partially_paid': [('IP_INSTALMENTS_PARTIALLY_PAID_SUM', 'sum')],
    'instalments_overdue': [('IP_INSTALMENTS_OVERDUE_SUM', 'sum')]
}

# Aplanar el diccionario para la agregación
flat_agg_dict = {}
for col, operations in agg_dict.items():
    for new_name, func in operations:
        flat_agg_dict[new_name] = (col, func)

# Aplicar la agregación con renombrado
installments_payments_df_agg = installments_payments_df.groupby(['SK_ID_CURR']).agg(**flat_agg_dict).reset_index()

Importamos datos de credit_card_balance

In [17]:
credit_card_balance_df = pd.read_csv("credit_card_balance.csv")

# Info sobre las columnas del credit_card_balance_df
"""
Histórico mensual del comportamiento de las tarjetas de crédito.
•	SK_ID_PREV → ID del crédito previo (relacionado a tarjeta).
•	SK_ID_CURR → ID del préstamo en nuestro dataset principal.
•	MONTHS_BALANCE → Mes relativo a la aplicación actual (0 = aplicación, -1 = un mes antes).
•	AMT_BALANCE → Balance actual de la tarjeta ese mes.
•	AMT_CREDIT_LIMIT_ACTUAL → Límite de crédito vigente ese mes.
•	AMT_DRAWINGS_ATM_CURRENT → Monto retirado en cajero automático ese mes.
•	AMT_DRAWINGS_CURRENT → Monto total retirado ese mes (ATM + POS + otros).
•	AMT_DRAWINGS_OTHER_CURRENT → Monto retirado en canales distintos a cajero o POS.
•	AMT_DRAWINGS_POS_CURRENT → Monto gastado en POS (compras).
•	AMT_INST_MIN_REGULARITY → Pago mínimo requerido ese mes.
•	AMT_PAYMENT_CURRENT → Pago realizado ese mes.
•	AMT_PAYMENT_TOTAL_CURRENT → Pagos totales realizados ese mes (suma de todos los abonos).
•	AMT_RECEIVABLE_PRINCIPAL → Principal pendiente de pago.
•	AMT_RECIVABLE → Total pendiente (principal + intereses).
•	AMT_TOTAL_RECEIVABLE → Variante de cálculo del total pendiente.
•	CNT_DRAWINGS_ATM_CURRENT → Número de retiros en cajero ese mes.
•	CNT_DRAWINGS_CURRENT → Número total de operaciones con la tarjeta ese mes.
•	CNT_DRAWINGS_OTHER_CURRENT → Número de operaciones en otros canales.
•	CNT_DRAWINGS_POS_CURRENT → Número de operaciones de compra en POS.
•	CNT_INSTALMENT_MATURE_CUM → Número acumulado de cuotas ya pagadas.
•	NAME_CONTRACT_STATUS → Estado del contrato (ej. Active, Completed).
•	SK_DPD → Días de atraso ese mes.
•	SK_DPD_DEF → Días de atraso con criterio más estricto (ignora deudas pequeñas).
"""
df_info_summary(credit_card_balance_df)

                            Non-Null Count  Null Count  % Null    Dtype
SK_ID_PREV                         3840312           0    0.00    int64
SK_ID_CURR                         3840312           0    0.00    int64
MONTHS_BALANCE                     3840312           0    0.00    int64
AMT_BALANCE                        3840312           0    0.00  float64
AMT_CREDIT_LIMIT_ACTUAL            3840312           0    0.00    int64
AMT_DRAWINGS_ATM_CURRENT           3090496      749816   19.52  float64
AMT_DRAWINGS_CURRENT               3840312           0    0.00  float64
AMT_DRAWINGS_OTHER_CURRENT         3090496      749816   19.52  float64
AMT_DRAWINGS_POS_CURRENT           3090496      749816   19.52  float64
AMT_INST_MIN_REGULARITY            3535076      305236    7.95  float64
AMT_PAYMENT_CURRENT                3072324      767988   20.00  float64
AMT_PAYMENT_TOTAL_CURRENT          3840312           0    0.00  float64
AMT_RECEIVABLE_PRINCIPAL           3840312           0    0.00  

In [18]:
# Sumarizar credit_card_balance_df por SK_ID_CURR para tener una fila por cada préstamo corriente
# Definir el diccionario de agregación según la columna
agg_dict = {
    # Columnas con media y renombrado
    'AMT_BALANCE': [('CCB_AMT_BALANCE_MEAN', 'mean')], # CCB_ para indicar que es de credit_card_balance
    'AMT_CREDIT_LIMIT_ACTUAL': [('CCB_AMT_CREDIT_LIMIT_ACTUAL_MEAN', 'mean')],
    'AMT_DRAWINGS_ATM_CURRENT': [('CCB_AMT_DRAWINGS_ATM_CURRENT_MEAN', 'mean')],
    'AMT_DRAWINGS_CURRENT': [('CCB_AMT_DRAWINGS_CURRENT_MEAN', 'mean')],
    'AMT_DRAWINGS_OTHER_CURRENT': [('CCB_AMT_DRAWINGS_OTHER_CURRENT_MEAN', 'mean')],
    'AMT_DRAWINGS_POS_CURRENT': [('CCB_AMT_DRAWINGS_POS_CURRENT_MEAN', 'mean')],
    'AMT_INST_MIN_REGULARITY': [('CCB_AMT_INST_MIN_REGULARITY_MEAN', 'mean')],
    'AMT_PAYMENT_CURRENT': [('CCB_AMT_PAYMENT_CURRENT_MEAN', 'mean')],
    'AMT_PAYMENT_TOTAL_CURRENT': [('CCB_AMT_PAYMENT_TOTAL_CURRENT_MEAN', 'mean')],
    'AMT_RECEIVABLE_PRINCIPAL': [('CCB_AMT_RECEIVABLE_PRINCIPAL_MEAN', 'mean')],
    'AMT_RECIVABLE': [('CCB_AMT_RECIVABLE_MEAN', 'mean')],
    'AMT_TOTAL_RECEIVABLE': [('CCB_AMT_TOTAL_RECEIVABLE_MEAN', 'mean')],
    'CNT_DRAWINGS_ATM_CURRENT': [('CCB_CNT_DRAWINGS_ATM_CURRENT_MEAN', 'mean')],
    'CNT_DRAWINGS_CURRENT': [('CCB_CNT_DRAWINGS_CURRENT_MEAN', 'mean')],
    'CNT_DRAWINGS_OTHER_CURRENT': [('CCB_CNT_DRAWINGS_OTHER_CURRENT_MEAN', 'mean')],
    'CNT_DRAWINGS_POS_CURRENT': [('CCB_CNT_DRAWINGS_POS_CURRENT_MEAN', 'mean')],
    'CNT_INSTALMENT_MATURE_CUM': [('CCB_CNT_INSTALMENT_MATURE_CUM_MEAN', 'mean')],
    'SK_DPD': [('CCB_SK_DPD_MEAN', 'mean')],
    'SK_DPD_DEF': [('CCB_SK_DPD_DEF_MEAN', 'mean')]
}

# Aplanar el diccionario para la agregación
flat_agg_dict = {}
for col, operations in agg_dict.items():
    for new_name, func in operations:
        flat_agg_dict[new_name] = (col, func)

# Aplicar la agregación con renombrado
credit_card_balance_df_agg = credit_card_balance_df.groupby(['SK_ID_CURR']).agg(**flat_agg_dict).reset_index()

# Mostrar las primeras filas
credit_card_balance_df_agg.head()

,SK_ID_CURR,CCB_AMT_BALANCE_MEAN,CCB_AMT_CREDIT_LIMIT_ACTUAL_MEAN,CCB_AMT_DRAWINGS_ATM_CURRENT_MEAN,CCB_AMT_DRAWINGS_CURRENT_MEAN,CCB_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CCB_AMT_DRAWINGS_POS_CURRENT_MEAN,CCB_AMT_INST_MIN_REGULARITY_MEAN,CCB_AMT_PAYMENT_CURRENT_MEAN,CCB_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CCB_AMT_RECEIVABLE_PRINCIPAL_MEAN,CCB_AMT_RECIVABLE_MEAN,CCB_AMT_TOTAL_RECEIVABLE_MEAN,CCB_CNT_DRAWINGS_ATM_CURRENT_MEAN,CCB_CNT_DRAWINGS_CURRENT_MEAN,CCB_CNT_DRAWINGS_OTHER_CURRENT_MEAN,CCB_CNT_DRAWINGS_POS_CURRENT_MEAN,CCB_CNT_INSTALMENT_MATURE_CUM_MEAN,CCB_SK_DPD_MEAN,CCB_SK_DPD_DEF_MEAN
0,100006,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
1,100011,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.0,3956.221849,4843.064189,4520.067568,52402.088919,54433.179122,54433.179122,0.054054,0.054054,0.0,0.0,25.767123,0.000000,0.000000
2,100013,18159.919219,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346250,6817.172344,17255.559844,18101.079844,18101.079844,0.255556,0.239583,0.0,0.0,18.719101,0.010417,0.010417
3,100021,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
4,100023,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000


### importamos aplication_train.csv y joineamos

In [19]:
train_df = pd.read_csv("application_train.csv")
len(train_df)

307511

In [20]:
len(bureau_df_agg)

305811

In [21]:
# Join train_df con bureau_df
train_df_join = train_df.merge(bureau_df_agg, on="SK_ID_CURR", how="left")

In [23]:
print(len(previous_application_agg))
print(len(train_df_join))

338857
307511


In [24]:
# Join train_df con previous_application_df
train_df_join = train_df_join.merge(previous_application_agg, on="SK_ID_CURR", how="left")

In [26]:
print(len(train_df_join))
print(len(pos_cash_balance_df_agg))

307511
337252


In [27]:
# Join train_df con pos_cash_balance_df
train_df_join = train_df_join.merge(pos_cash_balance_df_agg, on="SK_ID_CURR", how="left")

In [36]:
print(len(train_df_join))
print((len(credit_card_balance_df_agg)))

307511
103558


In [ ]:
# Join train_df con credit_card_balance_df ## Me falta hacer correr Credit Card Balance & installments payments
train_df_join = train_df_join.merge(credit_card_balance_df, on="SK_ID_CURR", how="left")


MemoryError: Unable to allocate 395. MiB for an array with shape (15, 3448571) and data type float64

In [ ]:
# Join train_df con installments_payments_df
train_df_join = train_df_join.merge(installments_payments_df, on="SK_ID_CURR", how="left")

In [ ]:
df_info_summary(train_df_join)

### Exportación

In [37]:
import os, re

def notebook_slug(default="notebook"):
    name = (os.environ.get("VSCODE_NOTEBOOK_FILENAME")
            or os.environ.get("NOTEBOOK_NAME"))
    if name:
        name = os.path.splitext(os.path.basename(name))[0]
    else:
        # 2) ipynbname si está instalado
        try:
            import ipynbname
            name = ipynbname.name()
        except Exception:
            name = default
    # sanitizar
    return re.sub(r'[^A-Za-z0-9_-]+', '_', str(name))

fname = f"application_train_{notebook_slug()}.csv"
train_df.to_csv(fname, index=False)
print("Guardado:", fname)

Guardado: application_train_notebook.csv
